### Data Merge - Batch
* Online Sample Data: 477420204194, 474620226651

#### 할일
* STEP-01. 각 데이터 타입별 대상 레코드 및 컬럼 추출
* STEP-02. 공사비별 전주/전선/인입선 갯 수 계산
* STEP-0x. 컬럼 추가(생성)
* STEP-0x. 전주 순서 계산

In [1]:
import os
import sys

import aidd.sys.config as cfg
from aidd.batch.data_manager import get_provide_data, read_data

In [2]:
pdata = get_provide_data()

[38cb1c83ad54][2024-03-12 20:00:35.687429] 제공받은 데이터 불러오기 시작
[38cb1c83ad54][2024-03-12 20:00:54.724610]   읽은 데이터: CONS, 데이터 크기: (19052, 143), 처리시간: 0:00:19.037123
[38cb1c83ad54][2024-03-12 20:01:12.224971]   읽은 데이터: POLE, 데이터 크기: (38533, 63), 처리시간: 0:00:17.500285
[38cb1c83ad54][2024-03-12 20:01:33.899514]   읽은 데이터: LINE, 데이터 크기: (40019, 77), 처리시간: 0:00:21.674465
[38cb1c83ad54][2024-03-12 20:01:42.847396]   읽은 데이터: SL, 데이터 크기: (22632, 57), 처리시간: 0:00:08.947801
[38cb1c83ad54][2024-03-12 20:01:42.847465] 제공받은 데이터 불러오기 종료, 최종 처리시간: 0:01:07.160038


#### 학습대상 추출
각 데이터별로 학습대상 레코드와 컬럼 추출

In [6]:
# 공사비 데이터 학습대상 레코드/컬럼 추출
df = pdata['CONS']
## [전처리]: 조건 적용(이 조건을 만족하지 않으면 제외), 
##           단, 실시간 처리시 총 공사비는 제외
modeling_records = \
    (df.ACC_TYPE_NAME == cfg.COND_ACC_TYPE_NAME) & \
    (df.CONT_CAP < cfg.COND_MAX_CONT_CAP) & \
    (df.CONS_TYPE_CD == cfg.COND_CONS_TYPE_CD) & \
    (df.TOTAL_CONS_COST < cfg.COND_MAX_TOTAL_CONS_COST)
cons_df = df[modeling_records].reset_index(drop=True)
cons_df = cons_df[cfg.CONS_COLS].copy()

print(f'공사비 데이터: 원본-{df.shape}, 학습대상-{cons_df.shape}')

공사비 데이터: 원본-(19052, 143), 학습대상-(15335, 8)


In [24]:
# 전주 데이터 학습대상 레코드/컬럼 추출
df = pdata['POLE']
pole_df = df[df.CONS_ID.isin(cons_df.CONS_ID)]
pole_df = pole_df[cfg.POLE_COLS]

print(f'전주 데이터: 원본-{df.shape}, 학습대상-{pole_df.shape}')

전주 데이터: 원본-(38533, 63), 학습대상-(27896, 6)


In [25]:
# 전선 데이터 학습대상 레코드/컬럼 추출
df = pdata['LINE']
line_df = df[df.CONS_ID.isin(cons_df.CONS_ID)]
line_df = line_df[cfg.LINE_COLS]

print(f'전선 데이터: 원본-{df.shape}, 학습대상-{line_df.shape}')

전선 데이터: 원본-(40019, 77), 학습대상-(30348, 11)


In [26]:
# 인입선 데이터 학습대상 레코드/컬럼 추출
df = pdata['SL']
sl_df = df[df.CONS_ID.isin(cons_df.CONS_ID)]
sl_df = sl_df[cfg.SL_COLS]

print(f'인입선 데이터: 원본-{df.shape}, 학습대상-{sl_df.shape}')

인입선 데이터: 원본-(22632, 57), 학습대상-(17034, 6)


In [28]:
sl_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17034 entries, 17 to 22631
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CONS_ID     17034 non-null  object 
 1   COMP_ID     17034 non-null  object 
 2   SL_TYPE_CD  17034 non-null  object 
 3   SL_SPEC_CD  17034 non-null  float64
 4   SPAN        17034 non-null  int64  
 5   SUPERVISOR  17034 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 931.5+ KB


In [29]:
df = read_data('PP', 'SL')

/home/freeman/projects/aidd/batch/data_manager.py:34: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, **kwargs)


In [30]:
df.shape

(11384, 157)

In [31]:
df[df.SL_CNT==0].sample(5)

,CONS_ID,TOTAL_CONS_COST,LAST_MOD_DATE,LAST_MOD_EID,OFFICE_NAME,EID_CODE,CONT_CAP,YEAR,MONTH,DAY,...,SL_SPEC_10.0,SL_SPEC_16.0,SL_SPEC_22.0,SL_SPEC_25.0,SL_SPEC_35.0,SL_SPEC_38.0,SL_SPEC_60.0,SL_SPEC_70.0,SL_SPEC_100.0,SL_SPEC_120.0
5456,475920223725,2176378,2022-11-24 10:42:22,MDP2100032,음성지사,MDP,3,2022,11,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1513,442720224406,3869929,2022-07-04 18:11:07,MDO2100010,동청주지사,MDO,3,2022,7,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6714,474920225034,2775023,2023-02-09 16:16:05,MDP2112857,제천지사,MDP,3,2023,2,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7320,70HC20214015,2285533,2022-01-19 09:28:28,MDE1503586,증평괴산지사,MDE,3,2022,1,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,477420223953,2316850,2022-12-28 08:37:43,MDP2100146,단양지사,MDP,5,2022,12,28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
